# 1. Introduction

## 1.1 Problem Statement


We are small investors and plan to open a restaurant in Ho Chi Minh City. In order to optimize the selection of location and scale of our restaurant, we shall consider the below criteria:
- Competitiveness: The neighborhood is not so competitive given (i) there are not too many restaurants in the neighborhood or (ii) the restaurants in the neighborhood are not highly rated in Foursquare.

- Neighborhood's population: The bigger population the better as we have a larger pool of potential customers. We do not want to operate our restaurant in suburban areas where there are not too many competitors but a small pool of potential customers as well.

- The restaurant tier (cheap, medium or expensive restaurant): This depends on the neighborhood's preference. We cannot open an highly expensive restaurant in a low-middle neighborhood and vice versa.

## 1.2. Data

Based on the criteria pre-defined above, we need the following data to perform the task:

- List of suggested venues from Foursquare API containing (i) venue's coordinates, (ii) venue's rating, and (iii) venue's tier.

- Population at district level.


Data source:

- Foursquare: Use regular calls to get suggested venues and their location. Use premimum calls to get venue's rating and tier.

- Citipopulation.de to get population at district level.

## 1.3. Methodology

We will use a combination of clustering models and map visualization to do the analyses.

In [1]:
import folium
import json
# import matplotlib.cm as cm
# import matplotlib.colors as colors
# import matlibplot.pyplot as plt
import numpy as np
import pandas as pd
# import seaborn as sns
import requests
from sklearn.cluster import KMeans

pd.set_option('display.max_rows', None)

The section below is copied from https://cocl.us/coursera_capstone_notebook project. Credit to the unknown author.

In [2]:
hcm_center = (10.762622, 106.660172)

In [3]:
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=48, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=48, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

In [4]:
hcm_center_x, hcm_center_y = lonlat_to_xy(hcm_center[1], hcm_center[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = hcm_center_x - 6000
x_step = 600
y_min = hcm_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []

for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(hcm_center_x, hcm_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

In [82]:
map_hcm = folium.Map(location=hcm_center, tiles='CartoDB dark_matter', zoom_start=13)

folium.Marker(hcm_center, popup='HCM').add_to(map_hcm)

for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_hcm) 
    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_hcm)
    #folium.Marker([lat, lon]).add_to(map_hcm)

map_hcm

In [6]:
def get_exploring_urls(client_id, client_secret, version, latitidues, longitudes, category, radius=600, limit=100, time='any', day='any'):
    urls = [f'https://api.foursquare.com/v2/venues/explore?&client_id={client_id}&client_secret={client_secret}&v={version}&ll={lat},{long}&categoryId={category}&radius={radius}&limit={limit}&time={time}&day={day}' for lat, long in zip(latitudes, longitudes)]

    return urls


def get_venue_details_urls(client_id, client_secret, version, venue_ids):
    urls = [f'https://api.foursquare.com/v2/venues/{venue_id}?client_id={client_id}&client_secret={client_secret}&v={20200101}' for venue_id in venue_ids]

    return urls


def get_venue_tips_urls(client_id, client_secret, version, venue_ids, limit=500):
    urls = [f'https://api.foursquare.com/v2/venues/{venue_id}/tips?client_id={client_id}client_secret={client_secret}&v={version}&limit={limit}' for venue_id in venue_ids]

    return urls


def get_venue_menu_urls(client_id, client_secret, version, venue_ids):
    urls = [f'https://api.foursquare.com/v2/venues/venue_id/menu?client_id={client_id}&client_secret={client_secret}&v={version}' for venue_id in venue_ids]

    return urls


def request_api(urls):
    results = [requests.get(url).json() for url in urls]

    return results


def check_errors(api_results):
    errors = []
    for idx, result in enumerate(api_results):
        if result['meta']['code'] != 200:
            errors[version].append((idx, result))
    
    return errors


def remedy_errors(api_results, urls, errors, tries=5):
    while len(errors) > 0 and tries > 0:
        for error in errors:
            idx = error[0]
            url = urls[idx]
            
            new_result = requests.get(url).json()
            api_results[idx] = new_result
            
        errors = check_errors(api_results)
        tries -= 1
    
    return api_results


def get_venues(api_results):
    venues = []
    for result in api_results:
        if result['meta']['code'] == 200:
            for item in result['response']['groups'][0]['items']:
                venues.append(item['venue'])
    
    return venues


def get_distinct_venues(venues):
    venue_ids = []
    distinct_venues = []
    for venue in venues:
        venue_id = venue['id']
        if venue_id not in venue_ids:
            venue_ids.append(venue_id)
            distinct_venues.append(venue)
    
    return distinct_venues


def get_venue_coordinates(venues):
    coordinates = [(venue['location']['lat'], venue['location']['lng']) for venue in venues]

    return coordinates


def get_venues_tier(venues_details):
    venue_tier = []
    for venue in venues_details:
        try:
            tier = venue['response']['venue']['price']['tier']
        except:
            tier = None

        venue_tier.append(tier)

    return venue_tier


def get_venues_likes(venues_details):
    venue_likes = []
    for venue in venues_details:
        try:
            likes = venue['response']['venue']['likes']['count']
        except:
            likes = None
        
        venue_likes.append(likes)
    
    return venue_likes


def get_venues_rating(venues_details):
    venue_rating = []
    for venue in venues_details:
        try:
            rating = venue['response']['venue']['rating']
        except:
            rating = None
        
        venue_rating.append(rating)
    
    return venue_rating

In [7]:
client_id = 'VBT1KEJNYQMDORW3N55MCKB3S35RZJVEG3I42IN3SSCLMQZO' # your Foursquare ID
client_secret = 'ZW3MU1JZLNMKMIS4NBS1X4VPSMMGXRIW1LLXL5KC0AGEZMXY' # your Foursquare Secret
versions = '20200101' # Foursquare API version

In [66]:
# food_category = '4d4b7105d754a06374d81259'

# urls_food = get_exploring_urls(client_id, client_secret, versions, latitudes, longitudes, food_category)
# results_food = request_api(urls_food)
# errors_food = check_errors(results_food)
# results_food = remedy_errors(results_food, urls_food, errors_food, tries=10)
# print('Errors count:', len(check_errors(results_food)))

In [251]:
# venues_food = get_distinct_venues(get_venues(results_food))

# with open('results_food.json', 'w') as fp:
#     json.dump(results_food, fp)

# with open('venues_food.json', 'w') as fp:
#     json.dump(venues_food, fp)

In [ ]:
# nightlife_category = '4d4b7105d754a06376d81259'

# urls_nightlife = get_exploring_urls(client_id, client_secret, versions, latitudes, longitudes, nightlife_category)
# results_nightlife = request_api(urls_nightlife)
# errors_nightlife = check_errors(results_nightlife)
# results_nightlife = remedy_errors(results_nightlife, urls_nightlife, errors_nightlife, tries=10)
# print('Errors count:', len(check_errors(results_nightlife)))

In [252]:
# venues_nightlife = get_distinct_venues(get_venues(results_nightlife))

# with open('results_nightlife.json', 'w') as fp:
#     json.dump(results_nightlife, fp)

# with open('venues_nightlife.json', 'w') as fp:
#     json.dump(venues_nightlife, fp)

In [8]:
# Load json files

with open('results_food.json', 'r') as f:
    results_food = json.load(f)

with open('venues_food.json', 'r') as f:
    venues_food = json.load(f)

with open('venues_details_food.json', 'r') as f:
    venues_details_food = json.load(f)

In [648]:
# start_idx = len(venues_details_food)
# end_idx = start_idx + 500

# temp_venues_id_food = [venue['id'] for venue in venues_food[start_idx:end_idx]]
# temp_venues_details_urls_food = get_venue_details_urls(client_id, client_secret, version, temp_venues_id_food)
# temp_venues_details_food = request_api(temp_venues_details_urls_food)
# venues_details_food.extend(temp_venues_details_food)

# with open('venues_details_food.json', 'w') as fp:
#     json.dump(venues_details_food, fp)

In [705]:
# for idx, venue in enumerate(venues_details_food):
#     if venue['meta']['code'] != 200:
#         venue_id = venues_food[idx]['id']
#         temp_url = get_venue_details_urls(client_id, client_secret, version, [venue_id])
#         temp_api_result = request_api(temp_url[0])
#         venue = temp_api_result[0]

In [11]:
from folium.plugins import MarkerCluster

venues_coor_food = get_venue_coordinates(venues_food)
coordinates = venues_coor_food.copy()

map_hcm = folium.Map(location=hcm_center,  tiles='CartoDB dark_matter', zoom_start=12)

marker_cluster = MarkerCluster().add_to(map_hcm)

for lat_long in coordinates:
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_hcm) 
    # folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_hcm)
    folium.Marker(lat_long).add_to(marker_cluster)

map_hcm

In [12]:
venues_tier = get_venues_tier(venues_details_food)
venues_likes = get_venues_likes(venues_details_food)
venues_rating = get_venues_rating(venues_details_food)

In [13]:
venues_id_food = [venue['id'] for venue in venues_food]

venue_coords_food = get_venue_coordinates(venues_food)
transposed_venue_coords_food = np.array(venue_coords_food).transpose()
lats = transposed_venue_coords_food[0]
longs = transposed_venue_coords_food[1]
x, y = lonlat_to_xy(longs[:967], lats[:967])

venues_df = pd.DataFrame({
    'venue_id': venues_id_food[:967],
    'latitude': lats[:967],
    'longitude': longs[:967],
    'x': x,
    'y': y,
    'tier': venues_tier,
    'likes': venues_likes,
    'rating': venues_rating
})
venues_df.set_index('venue_id', inplace=True)
venues_df.describe()

,latitude,longitude,x,y,tier,likes,rating
count,967.000000,967.000000,967.000000,9.670000e+02,917.000000,964.000000,482.000000
mean,10.755178,106.674134,683062.519029,1.189409e+06,1.635769,9.637967,6.821162
std,0.014433,0.027073,2959.167782,1.600158e+03,0.630681,22.827373,0.735305
min,10.709257,106.604434,675429.144665,1.184310e+06,1.000000,0.000000,5.200000
25%,10.744547,106.655931,681081.461884,1.188216e+06,1.000000,1.000000,6.200000
50%,10.758977,106.681192,683841.640981,1.189835e+06,2.000000,4.000000,6.800000
75%,10.767364,106.694843,685320.795592,1.190759e+06,2.000000,9.000000,7.300000
max,10.776120,106.715883,687631.440877,1.191687e+06,4.000000,376.000000,8.700000


In [14]:
X = venues_df[['x', 'y', 'tier', 'likes', 'rating']]
X.dropna(inplace=True)
X.describe()

,x,y,tier,likes,rating
count,458.000000,4.580000e+02,458.000000,458.000000,458.000000
mean,684495.510505,1.190100e+06,1.689956,18.144105,6.809607
std,1760.267409,1.302629e+03,0.637922,30.181453,0.736331
min,676327.034915,1.186546e+06,1.000000,2.000000,5.200000
25%,683788.077658,1.189688e+06,1.000000,6.000000,6.200000
50%,684839.224830,1.190605e+06,2.000000,10.000000,6.800000
75%,685781.353972,1.191021e+06,2.000000,18.750000,7.300000
max,687285.104009,1.191336e+06,4.000000,376.000000,8.700000


In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_X = scaler.fit_transform(X)
scaled_X

array([[ 0.33916722, -2.70640178,  0.48655252, -0.53548642, -2.18837353],
       [ 0.5919823 , -2.70827797, -1.08275067, -0.43597894, -0.69284642],
       [-0.91401902, -2.59166323,  0.48655252,  0.29374262,  0.12289564],
       ...,
       [ 1.07954859,  0.87828517,  0.48655252, -0.33647145, -1.50858848],
       [ 1.02405546,  0.93435972,  0.48655252, -0.4691481 , -1.37263147],
       [ 1.06649321,  0.90104961,  0.48655252, -0.43597894, -1.64454549]])

In [60]:
from sklearn.cluster import DBSCAN

db = DBSCAN(eps=0.3, min_samples=4).fit(scaled_X)
X['db_labels'] = db.labels_
X.groupby('db_labels')[['x', 'y', 'tier', 'likes', 'rating']].describe().stack()

x             y        tier       likes  \
db_labels                                                              
-1        count     392.000000  3.920000e+02  392.000000  392.000000   
          mean   684392.314506  1.190003e+06    1.681122   19.910714   
          std      1863.538736  1.344365e+03    0.665443   32.267420   
          min    676327.034915  1.186546e+06    1.000000    2.000000   
          25%    683486.008848  1.189498e+06    1.000000    7.000000   
          50%    684742.429024  1.190488e+06    2.000000   11.000000   
          75%    685849.629752  1.190998e+06    2.000000   21.000000   
          max    687285.104009  1.191336e+06    4.000000  376.000000   
 0        count       4.000000  4.000000e+00    4.000000    4.000000   
          mean   686057.911358  1.187789e+06    2.000000    6.750000   
          std       216.574185  4.077445e+01    0.000000    1.500000   
          min    685755.160788  1.187751e+06    2.000000    5.000000   
          25%    685975.799330  1.187759e+06    2.000000    5.750000   
          50%    686123.372869  1.187783e+06    2.000000    7.000000   
          75%    686205.484897  1.187813e+06    2.000000    8.000000   
          max    686229.738906  1.187840e+06    2.000000    8.000000   
 1        count       7.000000  7.000000e+00    7.000000    7.000000   
          mean   684442.383793  1.190189e+06    2.000000    4.714286   
          std       220.223825  2.867826e+02    0.000000    1.496026   
          min    684017.344146  1.189837e+06    2.000000    3.000000   
          25%    684385.004550  1.189965e+06    2.000000    3.500000   
          50%    684486.447161  1.190255e+06    2.000000    5.000000   
          75%    684557.892638  1.190334e+06    2.000000    5.500000   
          max    684707.100871  1.190637e+06    2.000000    7.000000   
 2        count      28.000000  2.800000e+01   28.000000   28.000000   
          mean   685183.539521  1.191010e+06    2.000000    8.678571   
          std       469.784437  2.189677e+02    0.000000    3.115714   
          min    684320.731507  1.190590e+06    2.000000    4.000000   
          25%    684963.322773  1.190895e+06    2.000000    6.000000   
          50%    685166.085531  1.191015e+06    2.000000    8.000000   
          75%    685549.281873  1.191199e+06    2.000000   10.000000   
          max    685947.571089  1.191324e+06    2.000000   16.000000   
 3        count       4.000000  4.000000e+00    4.000000    4.000000   
          mean   683785.167817  1.190881e+06    1.000000    7.250000   
          std       262.164124  1.470654e+02    0.000000    2.362908   
          min    683458.259516  1.190736e+06    1.000000    4.000000   
          25%    683703.898783  1.190767e+06    1.000000    6.250000   
          50%    683791.148283  1.190874e+06    1.000000    8.000000   
          75%    683872.417317  1.190987e+06    1.000000    9.000000   
          max    684100.115187  1.191040e+06    1.000000    9.000000   
 4        count       5.000000  5.000000e+00    5.000000    5.000000   
          mean   684898.154227  1.190504e+06    2.000000    9.200000   
          std        60.291697  1.790863e+02    0.000000    3.492850   
          min    684830.731965  1.190304e+06    2.000000    5.000000   
          25%    684843.285764  1.190314e+06    2.000000    7.000000   
          50%    684909.358636  1.190612e+06    2.000000    9.000000   
          75%    684935.144469  1.190635e+06    2.000000   11.000000   
          max    684972.250302  1.190656e+06    2.000000   14.000000   
 5        count       5.000000  5.000000e+00    5.000000    5.000000   
          mean   684710.843681  1.191113e+06    1.000000    6.600000   
          std       284.018262  7.783411e+01    0.000000    3.781534   
          min    684466.653943  1.191002e+06    1.000000    3.000000   
          25%    684502.334218  1.191077e+06    1.000000    5.000000   
          50%    684622.222860  1.191115e+06    1.000000    6.000000 

In [62]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=6, random_state=0).fit(scaled_X)
X['kmeans_labels'] = kmeans.labels_
X.groupby('kmeans_labels')[['x', 'y', 'tier', 'likes', 'rating']].describe().stack()

x             y        tier       likes  \
kmeans_labels                                                              
0             count      92.000000  9.200000e+01   92.000000   92.000000   
              mean   684825.800823  1.190669e+06    1.000000   14.163043   
              std       918.622056  5.238263e+02    0.000000   13.987653   
              min    682706.517526  1.189381e+06    1.000000    2.000000   
              25%    684125.097934  1.190436e+06    1.000000    6.000000   
              50%    684672.014708  1.190777e+06    1.000000    9.000000   
              75%    685392.341016  1.191113e+06    1.000000   16.500000   
              max    686710.785568  1.191311e+06    1.000000   81.000000   
1             count     117.000000  1.170000e+02  117.000000  117.000000   
              mean   684871.905555  1.190752e+06    1.897436   28.247863   
              std       964.176344  5.424440e+02    0.498007   28.988079   
              min    681772.910292  1.188976e+06    1.000000    4.000000   
              25%    684362.510703  1.190521e+06    2.000000    9.000000   
              50%    684992.214139  1.190919e+06    2.000000   15.000000   
              75%    685584.659333  1.191197e+06    2.000000   37.000000   
              max    686694.255242  1.191336e+06    3.000000  143.000000   
2             count     123.000000  1.230000e+02  123.000000  123.000000   
              mean   684775.915406  1.190588e+06    2.235772   11.056911   
              std      1334.344298  5.606055e+02    0.463085    9.030278   
              min    680700.530091  1.189191e+06    2.000000    2.000000   
              25%    684146.649215  1.190237e+06    2.000000    5.000000   
              50%    684802.240145  1.190656e+06    2.000000    8.000000   
              75%    685943.569706  1.191062e+06    2.000000   14.000000   
              max    686823.658489  1.191316e+06    4.000000   48.000000   
3             count      65.000000  6.500000e+01   65.000000   65.000000   
              mean   681308.025028  1.189791e+06    1.276923   12.938462   
              std      1352.645744  7.859999e+02    0.450961    8.999786   
              min    676327.034915  1.187922e+06    1.000000    3.000000   
              25%    680738.309035  1.189267e+06    1.000000    7.000000   
              50%    681393.232909  1.189695e+06    1.000000   10.000000   
              75%    682072.884849  1.190288e+06    2.000000   17.000000   
              max    683301.296174  1.191298e+06    2.000000   44.000000   
4             count       3.000000  3.000000e+00    3.000000    3.000000   
              mean   684980.352564  1.190897e+06    1.666667  299.000000   
              std       107.425637  2.870819e+02    0.577350  102.239914   
              min    684856.498765  1.190688e+06    1.000000  183.000000   
              25%    684946.412332  1.190733e+06    1.500000  260.500000   
              50%    685036.325898  1.190777e+06    2.000000  338.000000   
              75%    685042.279464  1.191001e+06    2.000000  357.000000   
              max    685048.233029  1.191224e+06    2.000000  376.000000   
5             count      58.000000  5.800000e+01   58.000000   58.000000   
              mean   686164.774244  1.187156e+06    1.672414   10.413793   
              std       817.128819  5.739960e+02    0.603729    7.862937   
              min    682888.350026  1.186546e+06    1.000000    2.000000   
              25%    685882.054678  1.186633e+06    1.000000    5.250000   
              50%    686301.749116  1.186914e+06    2.000000    7.500000   
              75%    686753.263359  1.187727e+06    2.000000   12.750000   
              max    687285.104009  1.188554e+06    3.000000   39.000000   

                         rating  
kmeans_labels                    
0             count   92.000000  
              mean     6.468478  
              std      0.525593  
              min      5.200000  
              25%      6.100

In [20]:
joined_X = X.join(venues_df[['latitude', 'longitude']], how='left')

In [48]:
# from folium.plugins import BeautifyIcon

coordinates = joined_X[['latitude', 'longitude']].values.copy()
labels = joined_X['db_labels'].copy()

map_hcm = folium.Map(location=hcm_center,  tiles='CartoDB dark_matter', zoom_start=14)

# marker_point = BeautifyIcon(iconShape='circle-dot').add_to(map_hcm)

# for lat_long in coordinates:
#     folium.CircleMarker([lat, lon], radius=2, fill=True, fill_color='blue', fill_opacity=1).add_to(map_hcm)
#     # folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_hcm)
#     # folium.Marker(lat_long).add_to(marker_point)

# map_hcm

clustered_venues = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for idx, (lat, lng) in enumerate(coordinates):
    if labels[idx] == 2:
        color = 'orange'
    else:
        color = 'grey'

    clustered_venues.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color=None,
            fill=True,
            fill_color=color,
            fill_opacity=0.6
        )
    )

# add incidents to map
map_hcm.add_child(clustered_venues)

In [63]:
coordinates = joined_X[['latitude', 'longitude']].values.copy()
labels = joined_X['kmeans_labels'].copy()
colors_set = {
    0: 'grey',
    1: 'blue',
    2: 'grey',
    3: 'grey',
    4: 'grey',
    5: 'grey',
    6: 'grey',
    7: 'grey',
}

map_hcm = folium.Map(location=hcm_center,  tiles='CartoDB dark_matter', zoom_start=14)

# marker_point = BeautifyIcon(iconShape='circle-dot').add_to(map_hcm)

# for lat_long in coordinates:
#     folium.CircleMarker([lat, lon], radius=2, fill=True, fill_color='blue', fill_opacity=1).add_to(map_hcm)
#     # folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_hcm)
#     # folium.Marker(lat_long).add_to(marker_point)

# map_hcm

clustered_venues = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for idx, (lat, lng) in enumerate(coordinates):
    clustered_venues.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color=None,
            fill=True,
            fill_color=colors_set[labels[idx]],
            fill_opacity=0.6
        )
    )

# add incidents to map
map_hcm.add_child(clustered_venues)

In [80]:
coordinates = joined_X[['latitude', 'longitude']].values.copy()
rating = joined_X['rating'].copy()

map_hcm = folium.Map(location=hcm_center,  tiles='CartoDB dark_matter', zoom_start=14)

# marker_point = BeautifyIcon(iconShape='circle-dot').add_to(map_hcm)

# for lat_long in coordinates:
#     folium.CircleMarker([lat, lon], radius=2, fill=True, fill_color='blue', fill_opacity=1).add_to(map_hcm)
#     # folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_hcm)
#     # folium.Marker(lat_long).add_to(marker_point)

# map_hcm

folium.Circle(hcm_center, tooltip='radius 6,000', radius=6000, color='blue', fill=False).add_to(map_hcm)
folium.Marker(hcm_center, tooltip='City Centroid').add_to(map_hcm)
clustered_venues = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for idx, (lat, lng) in enumerate(coordinates):
    if rating[idx] >= 8:
        color = 'darkgreen'
    elif rating[idx] >= 7.5:
        color = 'green'
    elif rating[idx] >= 7:
        color = 'lightgreen'
    else:
        color = 'grey'

    clustered_venues.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color=None,
            fill=True,
            fill_color=color,
            fill_opacity=0.6
        )
    )

# add incidents to map
map_hcm.add_child(clustered_venues)